In [2]:
import json
import os
import random
import time

In [3]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [4]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [5]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [6]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [7]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [8]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [9]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Churches Data

In [16]:
def get_coords(url: str) -> Point:
    r = scraper.get(url)
    if r.status_code != 200:
        print("Status Code:", r.status_code, url)
        return {}
    try:
        soup_store = BeautifulSoup(r.text, "html.parser")
        store_json = json.loads(soup_store.find_all("script")[1].text)
        lat = float(store_json["@graph"][0]["geo"]["latitude"])
        lon = float(store_json["@graph"][0]["geo"]["longitude"])
    except Exception as e:
        print(e, url)
    return Point(lon, lat)

In [21]:
scraper = cloudscraper.create_scraper()

In [22]:
root_url = "https://locations.churchs.com/"

In [47]:
churches_list = []

root_url = "https://locations.churchs.com/"
r_all_states = scraper.get(root_url)
soup = BeautifulSoup(r_all_states.text, "html.parser")
state_as = soup.find_all(
    "a",
    {"class": "Link inline-block after:content-[attr(data-count)] after:ml-2 w-full"},
)

for state_a in tqdm(state_as, desc="Parsing States"):
    state_code = state_a.attrs["href"]
    state_store_list = []

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_url = os.path.join(root_url, state_code)

    r_state = scraper.get(state_url)
    soup_state = BeautifulSoup(r_state.text, "html.parser")
    location_as = soup_state.find_all(
        "a",
        {
            "class": "Link inline-block after:content-[attr(data-count)] after:ml-2 w-full"
        },
    )

    for location_a in tqdm(location_as, desc=f"Parsing Locations in {state_code}"):
        location_url = os.path.join(root_url, location_a.attrs["href"])
        r_location = scraper.get(location_url)
        soup_location = BeautifulSoup(r_location.text, "html.parser")
        store_as = soup_location.find_all("a", {"class": "font-bold underline"})
        store_as = [store_a for store_a in store_as if store_a.text == "Store Page"]

        for store_a in store_as:
            store_url = store_a.attrs["href"]
            point = get_coords(store_url)
            store_dict = {"STATE": state_code.upper(), "geometry": point}
            churches_list.append(store_dict)
            state_store_list.append(store_dict)

        time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        churches_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        churches_state_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/27 [00:00<?, ?it/s]

Parsing Locations in al:   0%|          | 0/22 [00:00<?, ?it/s]

Parsing Locations in ar:   0%|          | 0/8 [00:00<?, ?it/s]

Parsing Locations in az:   0%|          | 0/19 [00:00<?, ?it/s]

Parsing Locations in ca:   0%|          | 0/32 [00:00<?, ?it/s]

...

Parsing Locations in tn:   0%|          | 0/4 [00:00<?, ?it/s]

Parsing Locations in tx:   0%|          | 0/146 [00:00<?, ?it/s]

Parsing Locations in va:   0%|          | 0/4 [00:00<?, ?it/s]

Parsing Locations in wa:   0%|          | 0/5 [00:00<?, ?it/s]

Parsing Locations in wi:   0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
churches_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        churches_gdf = gpd.read_file(constructed_path)
        churches_gdfs.append(churches_gdf)

In [49]:
churches_gdf = gpd.GeoDataFrame(pd.concat(churches_gdfs, ignore_index=True))

In [51]:
churches_gdf = churches_gdf.to_crs(9311)
churches_gdf.to_file("data/churches.gpkg")

In [52]:
churches_counts_gdf = pd.DataFrame(
    churches_gdf.groupby("STATE").size(), columns=["CHURCHES"]
)
churches_counts_gdf = churches_counts_gdf.reset_index()
churches_counts_gdf = churches_counts_gdf.rename(columns={"STATE": "STUSPS"})

## Combine With States

In [53]:
churches_by_states_gdf = states_with_population_df.merge(
    churches_counts_gdf, on="STUSPS", how="left"
)

In [54]:
churches_by_states_gdf = churches_by_states_gdf.fillna(0)
churches_by_states_gdf["CHURCHES"] = churches_by_states_gdf["CHURCHES"].astype(int)

In [55]:
churches_by_states_gdf["per_1000"] = churches_by_states_gdf["CHURCHES"] / (
    churches_by_states_gdf["POPULATION"] / 1000
)
churches_by_states_gdf["per_10k"] = churches_by_states_gdf["CHURCHES"] / (
    churches_by_states_gdf["POPULATION"] / 10_000
)
churches_by_states_gdf["per_100k"] = churches_by_states_gdf["CHURCHES"] / (
    churches_by_states_gdf["POPULATION"] / 100000
)
churches_by_states_gdf["per_500k"] = churches_by_states_gdf["CHURCHES"] / (
    churches_by_states_gdf["POPULATION"] / 500_000
)
churches_by_states_gdf["per_1m"] = churches_by_states_gdf["CHURCHES"] / (
    churches_by_states_gdf["POPULATION"] / 1_000_000
)

In [56]:
churches_by_states_gdf = churches_by_states_gdf.dropna()

In [57]:
churches_by_states_gdf = churches_by_states_gdf.to_crs(9311)
churches_by_states_gdf.to_file("data/churches_per_state.gpkg")